In [51]:
import os
from tabulate import tabulate
from nilearn import plotting
from nilearn import image
from nilearn.image import math_img, resample_to_img, get_data
from nilearn.datasets import fetch_atlas_harvard_oxford
from nilearn.plotting import plot_stat_map, view_img
import numpy as np
from scipy.stats import ttest_1samp

In [ ]:
def get_subject_file_pairs(data_dir):
    file_pairs = []
    for subject in os.listdir(data_dir):
        subject_path = os.path.join(data_dir, subject)
        # Skip non-directories or irrelevant files
        if not os.path.isdir(subject_path) or subject.startswith('.'):
            continue
        # Locate BL and FU files
        baseline_file = os.path.join(subject_path, 'ses-BL', 'anat', f"{subject}_ses-BL_T1w.nii.gz")
        followup_file = os.path.join(subject_path, 'ses-FU', 'anat', f"{subject}_ses-FU_T1w.nii.gz")
        # Check if both files exist
        if os.path.exists(baseline_file) and os.path.exists(followup_file):
            # Append relative paths as a tuple
            file_pairs.append((os.path.relpath(baseline_file, data_dir),
                                os.path.relpath(followup_file, data_dir)))
        else:
            print(f"Skipping subject {subject}: missing files.")
    return file_pairs

In [55]:
# Fetch the Harvard-Oxford atlas
ATLAS = image.load_img(fetch_atlas_harvard_oxford('cort-maxprob-thr25-1mm'))
ATLAS_DATA = get_data(atlas_img)
# Extract region names from the atlas
LABEL_NAMES = ATLAS['labels']
# Extract unique region labels from the atlas
REGION_LABELS = np.unique(ATLAS_DATA)

def loadImage(imgPath):
    img = image.load_img(imgPath)
    img = resample_to_img(source_img=img, target_img=atlas_img, force_resample=True, copy_header=True)
    img = image.smooth_img(img1_resampled, fwhm=6)
    return img

def findDifferingAreas(img1, img2):
    # Creates the difference map
    diff = image.math_img("img1 - img2", img1=img1, img2=img2)
    diff_data = get_data(diff)
    # Prepares the output list
    regionList = []
    # Loop through each region to compute statistics
    for label in REGION_LABELS:
        regionMask = (ATLAS_DATA == label)
        regionChanges = diff_data[regionMask]
        regionList.append((LABEL_NAMES[label - 1], np.mean(regionChanges)))
    regionList.sort(reverse = True, key = lambda x: x[1])
    return regionList

# Image Paths
imagePath1 = "Data/sub-101/ses-BL/anat/sub-101_ses-BL_T1w.nii.gz"
imagePath2 = "Data/sub-101/ses-FU/anat/sub-101_ses-FU_T1w.nii.gz"
# Converts the given paths to images
baseLine = loadImage(imgPath1)
followUp = loadImage(imgPath2)
regionList = findDifferingAreas(img1, img2)


182

In [56]:
import os

def list_files(directory):
    for dirpath, dirnames, filenames in os.walk(directory):
        print(f"Directory: {dirpath}")
        for file in filenames:
            print(f"  File: {file}")

list_files('Data')

Directory: Data
  File: .DS_Store
  File: LICENSE
  File: CHANGES
  File: README
  File: dataset_description.json
  File: T1w.json
  File: participants.tsv
Directory: Data/sub-213
  File: .DS_Store
Directory: Data/sub-213/ses-BL
Directory: Data/sub-213/ses-BL/anat
  File: sub-213_ses-BL_T1w.nii.gz
Directory: Data/sub-213/ses-FU
Directory: Data/sub-213/ses-FU/anat
  File: sub-213_ses-FU_T1w.nii.gz
Directory: Data/sub-222
  File: .DS_Store
Directory: Data/sub-222/ses-BL
Directory: Data/sub-222/ses-BL/anat
  File: sub-222_ses-BL_T1w.nii.gz
Directory: Data/sub-222/ses-FU
Directory: Data/sub-222/ses-FU/anat
  File: sub-222_ses-FU_T1w.nii.gz
Directory: Data/sub-302
  File: .DS_Store
Directory: Data/sub-302/ses-BL
Directory: Data/sub-302/ses-BL/anat
  File: sub-302_ses-BL_T1w.nii.gz
Directory: Data/sub-302/ses-FU
Directory: Data/sub-302/ses-FU/anat
  File: sub-302_ses-FU_T1w.nii.gz
Directory: Data/sub-130
  File: .DS_Store
Directory: Data/sub-130/ses-BL
Directory: Data/sub-130/ses-BL/anat
  F